In [1]:
import tensorflow
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12526285747980507305
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 11332668621
locality {
  bus_id: 1
}
incarnation: 12664852060215299067
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:1e.0"
]


In [2]:
from keras import backend as K
import keras.utils.np_utils

from keras.models import Sequential
from keras.layers import Input
from keras.models import Model
from keras.layers.core import Flatten, Dense, Dropout, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D, ZeroPadding1D
from keras.applications.vgg16 import VGG16

from keras. preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
import numpy as np
import pandas as pd

from IPython.display import SVG
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import cv2
import sys
from IPython.display import display 
from keras.utils.training_utils import multi_gpu_model



#import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

Using Theano backend.
Using cuDNN version 5110 on context None
Mapped name None to device cuda: Tesla K80 (0000:00:1E.0)


In [3]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1/255, horizontal_flip=True)

In [6]:
def plot_images(x, y, **kwargs):
    n_pix = int(np.sqrt(np.prod(x.shape[1:3]))) #assumes images are square
    im_indices = np.random.choice(x.shape[0], 36, replace=False)
    fig, axes = subplots(nrows=6,ncols=6, figsize=(10,10), sharex=True, sharey=True, frameon=False)
    for i,ax in enumerate(axes.flat):
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
        curr_i = im_indices[i]
#         curr_label = label[argmax(y[curr_i])]
        ax.imshow(x[curr_i].reshape(n_pix,n_pix, 3), aspect="auto", **kwargs)
        ax.text(10,20,y[curr_i], fontdict={"backgroundcolor": "gray","color": "white" })
#         ax.set_title(title)
        ax.axis('off')
    plt.tight_layout(h_pad=0, w_pad=0)

In [22]:
%pylab inline
IMSIZE = [100, 100]
BATCH  =  32

train_gentrain_g  = datagen.flow_from_directory(class_mode='sparse',
    directory='images/imdb_crop/test_train/train',
    target_size=IMSIZE,
    batch_size=BATCH,
)

val_gen = datagen.flow_from_directory(class_mode='sparse',
    directory='images/imdb_crop/test_train/test',
    target_size=IMSIZE,
    batch_size=BATCH,
)
x,y = next(train_gentrain_g)

Populating the interactive namespace from numpy and matplotlib
Found 149530 images belonging to 101 classes.
Found 64025 images belonging to 101 classes.


In [16]:
plot_images(x, y, interpolation="spline16")

ValueError: Cannot take a larger sample than population when 'replace=False'

In [23]:
print(x.shape)

(32, 100, 100, 3)


In [24]:
x.shape[1:]

(100, 100, 3)

In [25]:
#Get back the convolutional part of a VGG network trained on ImageNet
model_vgg16_conv = VGG16(weights='imagenet',pooling='avg', include_top=False)


#model_vgg16_conv.summary()

# Tailor our input format (Note- Tensorflow input order!)
input = Input(shape=x.shape[1:],name = 'image_input')  # our input shape: (32,32,3)

#Use the generated model 
output_vgg16_conv = model_vgg16_conv(input)

#Add the fully-connected layers 
#x = Flatten(name='flatten')
x = Dense(100, activation='relu', name='fc1')(output_vgg16_conv)
x = Dense(50, activation='relu', name='fc2')(x)
x = Dense(1, activation='linear', name='predictions')(x)

#Create your own model 
my_model = Model(input=input, output=x)  # Using the Functional API
# Freeze convolutional layers
for layer in my_model.layers[:2]:
    layer.trainable=False

#In the summary, weights and layers from VGG part will be hidden, but they will be fit during the training
my_model.summary()

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=vgg16/bloc..., outputs=Elemwise{a...)`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_input (InputLayer)     (None, 100, 100, 3)       0         
_________________________________________________________________
vgg16 (Model)                (None, 512)               14714688  
_________________________________________________________________
fc1 (Dense)                  (None, 100)               51300     
_________________________________________________________________
fc2 (Dense)                  (None, 50)                5050      
_________________________________________________________________
predictions (Dense)          (None, 1)                 51        
Total params: 14,771,089
Trainable params: 56,401
Non-trainable params: 14,714,688
_________________________________________________________________


In [26]:
from sklearn.metrics import r2_score



In [27]:
def custom_metric(y_true, y_pred):
    return K.mean(K.square(y_true-y_pred))



In [28]:
# must use a very small learning rate 
sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)

my_model.compile(optimizer='nadam', loss='mean_squared_error')


In [ ]:
import glob

glob.glob('/home/ubuntu/project_pi/model/*')

In [29]:
# prepare data augmentation configuration 
test_steps = val_gen.n//val_gen.batch_size
train_steps = train_gentrain_g.n//train_gentrain_g.batch_size




# same callbacks!
cb_a=keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')
# lets look at our results while we are at it
cb_b=keras.callbacks.ReduceLROnPlateau(factor=.5, patience=2, min_lr=1e-8)
#cb_c = keras.callbacks.ModelCheckpoint('/home/ubuntu/project_pi/model/weights.hdf5', monitor='mean_squared_error', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)

# fine-tune the model
history = my_model.fit_generator(
    train_gentrain_g,
    steps_per_epoch= train_steps,
    nb_epoch=100,
    #validation_data=val_gen,
    #validation_steps=  test_steps,
    callbacks=[cb_b, cb_a])

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., steps_per_epoch=4672, callbacks=[<keras.ca..., epochs=100)`


Epoch 1/100
4672/4672 [==============================] - 702s 150ms/step - loss: 208.9102
Epoch 2/100
   1/4672 [..............................] - ETA: 11:27 - loss: 210.6452

/home/ubuntu/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:1043: RuntimeWarning: Reduce LR on plateau conditioned on metric `val_loss` which is not available. Available metrics are: loss,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning
/home/ubuntu/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:535: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


4672/4672 [==============================] - 703s 151ms/step - loss: 203.5789
Epoch 3/100
4672/4672 [==============================] - 704s 151ms/step - loss: 202.7502
Epoch 4/100
4672/4672 [==============================] - 704s 151ms/step - loss: 202.0841
Epoch 5/100
4672/4672 [==============================] - 704s 151ms/step - loss: 201.4522
Epoch 6/100
4672/4672 [==============================] - 704s 151ms/step - loss: 201.0056
Epoch 7/100
4672/4672 [==============================] - 703s 151ms/step - loss: 200.6410
Epoch 8/100
4672/4672 [==============================] - 704s 151ms/step - loss: 200.3866
Epoch 9/100
4672/4672 [==============================] - 704s 151ms/step - loss: 199.9775
Epoch 10/100
4672/4672 [==============================] - 711s 152ms/step - loss: 199.3928
Epoch 11/100
4672/4672 [==============================] - 715s 153ms/step - loss: 199.1468
Epoch 12/100
4672/4672 [==============================] - 704s 151ms/step - loss: 198.9758
Epoch 13/100
4672/4

KeyboardInterrupt: 

In [30]:
my_model.save('mon_model.h5')

In [ ]:
glob.glob('/home/ubuntu/project_pi/images/imdb_crop/test_train/test/0/*')

In [ ]:
my_model.save('model.hdf5')

In [ ]:


img = cv2.imread('/home/ubuntu/project_pi/images/imdb_crop/test_train/test/22/nm1065229_rm148540928_1984-4-18_2006.jpg',
)
img = cv2.resize(img,(120,120))
img = np.reshape(img,[1,120,120,3])

classes = model.predict(img)

print(classes)



In [ ]:
from keras.models import load_model


model = load_model('my_model1.h5')

In [ ]:
img = cv2.imread('/home/ubuntu/project_pi/images/imdb_crop/test_train/test/33/nm0000496_rm3777730816_1973-6-21_2006.jpg')
img = cv2.resize(img,(120,120))
img = np.reshape(img,[1,120,120,3])
 
predict = model.predict(img, verbose=1)

Z = [x for _,x in sorted(zip(label,predict[0]))]
argmax(Z)


In [ ]:
from IPython.display import Image
Image(filename='/home/ubuntu/project_pi/images/imdb_crop/test_train/test/33/nm0000496_rm3777730816_1973-6-21_2006.jpg') 

In [ ]:
img = cv2.imread('/home/ubuntu/project_pi/images/imdb_crop/test_train/test/0/nm0837177_rm1374792704_1975-9-18_1975.jpg')
img = cv2.resize(img,(120,120))
img = np.reshape(img,[1,120,120,3])
 
predict = model.predict(img, verbose=1)

Z = [x for _,x in sorted(zip(label,predict[0]))]
argmax(Z)


In [ ]:
from IPython.display import Image
Image(filename= '/home/ubuntu/project_pi/images/imdb_crop/test_train/test/0/nm0837177_rm1374792704_1975-9-18_1975.jpg')

In [5]:
### model 3 continue model fine tunning

#Get back the convolutional part of a VGG network trained on ImageNet
model_vgg16_conv = VGG16(weights='imagenet',pooling='avg', include_top=False)


#model_vgg16_conv.summary()

# Tailor our input format (Note- Tensorflow input order!)
input = Input(shape=(120,120,3),name = 'image_input')  # our input shape: (32,32,3)

#Use the generated model 
output_vgg16_conv = model_vgg16_conv(input)

#Add the fully-connected layers 
#x = Flatten(name='flatten')
x = Dense(500, activation='relu', name='fc1')(output_vgg16_conv)
x = Dense(500, activation='relu', name='fc2')(x)
x = Dense(1, activation='linear', name='predictions')(x)

#Create your own model 
my_model = Model(input=input, output=x)  # Using the Functional API
# Freeze convolutional layers
for layer in my_model.layers[:2]:
    layer.trainable=False

#In the summary, weights and layers from VGG part will be hidden, but they will be fit during the training
my_model.summary()

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=/image_inp..., outputs=Elemwise{a...)`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_input (InputLayer)     (None, 120, 120, 3)       0         
_________________________________________________________________
vgg16 (Model)                (None, 512)               14714688  
_________________________________________________________________
fc1 (Dense)                  (None, 500)               256500    
_________________________________________________________________
fc2 (Dense)                  (None, 500)               250500    
_________________________________________________________________
predictions (Dense)          (None, 1)                 501       
Total params: 15,222,189
Trainable params: 507,501
Non-trainable params: 14,714,688
_________________________________________________________________


In [6]:
def VGG_16(weights_path=None):
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(3,224,224)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(512,(3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(Flatten())
    model.add(Dense(500, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(500, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='linear'))

    if weights_path:
        model.load_weights(weights_path)

    return model

In [7]:
model_vgg16_two = VGG16(weights='imagenet',pooling='avg', include_top=False)

for layer in model_vgg16_two.layers[:20]:
    layer.trainable= False 
    
print(len(model_vgg16_two.layers))

20


In [8]:
top_model = Sequential()
top_model.add(Flatten(input_shape=[120,120,3]))
top_model.add(Dense(500, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(500, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='linear'))

In [9]:
# same callbacks!
cb_a=keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='auto')
# lets look at our results while we are at it
cb_b=keras.callbacks.ReduceLROnPlateau(factor=.5, patience=2, min_lr=1e-8)

In [ ]:
# prepare data augmentation configuration 
test_steps = val_gen.n//val_gen.batch_size
train_steps = train_gentrain_g.n//train_gentrain_g.batch_size

top_model.compile(optimizer='adam', loss='mean_squared_error')

top_model.fit_generator(
    train_gentrain_g,
    steps_per_epoch= train_steps,
    nb_epoch=100,
    validation_data=val_gen,
    validation_steps=  test_steps,
    callbacks=[cb_b, cb_a])

top_model.save('top_model.h5')

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  del sys.path[0]
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., steps_per_epoch=2336, validation_data=<keras_pre..., validation_steps=1000, callbacks=[<keras.ca..., epochs=100)`
  del sys.path[0]


Epoch 1/100
2336/2336 [==============================] - 385s 165ms/step - loss: 298.3438 - val_loss: 534.4996
Epoch 2/100
2336/2336 [==============================] - 384s 165ms/step - loss: 263.9224 - val_loss: 298.5476
Epoch 3/100
2336/2336 [==============================] - 384s 165ms/step - loss: 218.1231 - val_loss: 213.8861
Epoch 6/100
2336/2336 [==============================] - 385s 165ms/step - loss: 218.2181 - val_loss: 213.8560
